In [13]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [14]:
from fastai.text import *

/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [15]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.counter = collections.Counter()
        self.total = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1

        token_id = self.word2idx[word]
        self.counter[token_id] += 1
        self.total += 1
        return token_id

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))
        self.num_tokens = len(self.dictionary)

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids
    
# https://github.com/fastai/fastai_docs/blob/master/dev_nb/007_wikitext_2.ipynb
# download data from https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
EOS = '<eos>'
PATH=Path('')

def read_file(filename):
    tokens = []
    with open(PATH/filename, encoding='utf8') as f:
        for line in f:
            tokens.append(line.split() + [EOS])
    return np.array(tokens)

In [16]:
train_tok = read_file('train.txt')
valid_tok = read_file('valid.txt')
test_tok = read_file('test.txt')

# cor = Corpus('')
# voc = Vocab(cor.dictionary.idx2word)

In [5]:
len(voc.itos)

NameError: name 'voc' is not defined

In [ ]:
voc.itos

In [5]:
a = list(train_tok)

In [6]:
flat_list = [item for sublist in a for item in sublist]

In [7]:
len(flat_list)
# 600k tokens
# * 2 ~ ptb
# * 5 ~ wikitext-2
# * 300 ~ wikitext-103

602169

In [29]:
train_tok

array([list(['<eos>']), list(['<eos>']), list(['<eos>']), list(['<eos>']), ...,
       list(['Tel:', '(760)', '291–1003', '<eos>']), list(['Fax:', '(760)', '291–1172', '<eos>']),
       list(['Deerpark@plumvillage.org', '<eos>']), list(['<eos>'])], dtype=object)

In [42]:
36000*30

1080000

In [17]:
bs = 256
bptt = 35

In [21]:
dat = TextLMDataBunch.from_tokens('data', 
                                  trn_tok = train_tok, trn_lbls = None,
                                  val_tok = valid_tok, val_lbls=None,
                                  tst_tok = test_tok, 
#                                   vocab = voc,
                                  bs = bs, bptt = bptt)

In [10]:
# [] list has some method
# voc.itos is a list so it has those default method

In [73]:
dat.train_ds

LabelList (58665 items)
x: LMTextList
<eos>,<eos>,<eos>,<eos>,<eos>
y: LMLabelList
,,,,
Path: data

In [8]:
# pdftotext create incorrect positioned <eos> tokens
dat.show_batch()

idx,text
0,"cultivate mindfulness, we can do the same <eos> things we always xxunk sitting, working, <eos> eating, and so xxunk mindful awareness <eos> of what we are doing. When we’re eating, we <eos> know that we"
1,"is mindfulness and concentra- <eos> tion, there is insight. When you put that grain <eos> of rice into your mouth, you are putting the <eos> whole universe in your mouth. This is possible <eos> when"
2,"concentration. Your body is not just yours. It is <eos> a gift and a responsibility. To keep it healthy, <eos> we need to know how to eat. <eos> <eos> <eos> choosing what to eat <eos>"
3,"one hundred monks. It was a lot <eos> of fun washing dishes together, even without <eos> hot running water and xxunk Some countries <eos> have modern homes that are very comfortable. <eos> The water, hot"
4,"you will be <eos> worthy of the Earth and the sky. <eos> When we look deeply at the work that goes <eos> into growing and preparing our food, gratitude <eos> comes naturally. So many hands"


In [22]:
learn = language_model_learner(dat, AWD_LSTM, drop_mult=0.3)

In [23]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 

In [ ]:
learn.recorder.plot(skip_end=15)

In [24]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,6.067256,4.951650,0.189955,00:42


In [25]:
learn.unfreeze()

In [26]:
learn.fit_one_cycle(2, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.196928,4.755712,0.179855,00:55
1,4.988514,4.821382,0.159710,00:55


In [16]:
learn.export('..\viriya_deploy\tnh.pkl')

In [32]:
TEXT = "sfafa"
N_WORDS = 100
N_SENTENCES = 1

In [19]:
learn.beam_search('Anger is good.', n_words=20, beam_sz=20)

'Anger is good. n g e r xxunk i s xxunk g o o d . . . . . . . . . . . . . . . . . . . . .'

In [33]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

sfafa g o h g t l w o o o n d e l u d e s s c l e i d d u n e i n s n d i d s i d v i d e i n g i n g i n i n n g e i n n i d i s i n g a m i n e d i n i n i n e e i n s l u i n d s n s i s u i n e t e i s


In [37]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Anger Chapter West - said, words. The Buddha told the Buddha that the Buddha had preparing them for a special meal in the future. The Buddha said, Only our friend King Bimbisara is a great All a spiritual Rahula, and the senior bhikkhu in his nation would be the Buddha’s The
Anger you are a long time in your daily heart. If you are fully awakened and at that moment you are suddenly the same or Buddha, I will never forget that you are I take the time to tell you about the Buddha in the very nature of his own body,


In [3]:
import urllib.request


In [6]:
...
# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve(url='https://drive.google.com/uc?export=download&id=1aONdl2-puQYv6LylrPolIWKE269kj6mL')

('/tmp/tmpm4gg0br_', <http.client.HTTPMessage at 0x7fd91b4d4eb8>)

In [7]:
import urllib.request
import shutil

In [9]:

# Download the file from `url` and save it locally under `file_name`:
with urllib.request.urlopen(url='https://drive.google.com/uc?export=download&id=1aONdl2-puQYv6LylrPolIWKE269kj6mL') as response, open('t.pkl', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

In [4]:
?urllib.request.urlretrieve

Signature: urllib.request.urlretrieve(url, filename=None, reporthook=None, data=None)
Docstring:
Retrieve a URL into a temporary location on disk.

Requires a URL argument. If a filename is passed, it is used as
the temporary file location. The reporthook argument should be
a callable that accepts a block number, a read size, and the
total file size of the URL target. The data argument should be
valid URL encoded data.

If a filename is passed and the URL points to a local resource,
the result is a copy from local file to new file.

Returns a tuple containing the path to the newly created
data file as well as the resulting HTTPMessage object.
File:      /opt/anaconda3/lib/python3.7/urllib/request.py
Type:      function


In [ ]:

html = response.read()